In [1]:
import torch
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification

model_path = '/content/drive/My Drive/LIN371/classification_only_model'

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load model
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

In [3]:
def predict(model, inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(
        inputs,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        attention_mask=attention_mask
    )
    return output.logits  # For sequence classification


In [4]:
import torch.nn.functional as F

def perturb_sent(model, tokenizer, sent):
  inputs = tokenizer(
    [sent],
    return_tensors="pt",
    truncation=True,
    max_length=512,
    padding='max_length',
)

  input_ids = inputs["input_ids"]
  attention_mask = inputs["attention_mask"]

  replace_tokens = ['unmasked']

  # create new sentences
  # take original sentence and mask one token in order
  for tokenized_sent in input_ids:
    for i, token in enumerate(tokenized_sent):
      if token not in [tokenizer.pad_token_id, tokenizer.cls_token_id, tokenizer.sep_token_id]:
        new_sent = tokenized_sent.clone().detach()
        new_sent[i] = tokenizer.mask_token_id
        input_ids = torch.cat((input_ids, new_sent.unsqueeze(0)), dim=0)
        replace_tokens.append(tokenizer.decode(token))

  pred = predict(model, inputs=input_ids.to(device), attention_mask=attention_mask.to(device))

  probabilities = F.softmax(pred, dim=-1)

  # large difference in probabilities when masking means that the token is probably important
  # for the classification
  differences = (probabilities - probabilities[0])[:,0]

  max_len = max(len(token) for token in replace_tokens)

  print('sentence:', sent)
  print(f"{'token':^{max_len}} |",
        f"{'class 0 proba':^{13}} |",
        f"{'class 1 proba':^{13}} |",
        f"{'change (class 0)':^{16}} |")
  for probability, token, diff in zip(probabilities, replace_tokens, differences):
    class0 = probability[0]
    class1 = probability[1]
    print(f'{token:^{max_len}} |',
          f'{round(class0.item(), 3):^{13}} |',
          f'{round(class1.item(), 3):^{13}} |',
          f'{round(diff.item(), 3):^{16}} |')

  return probabilities, replace_tokens, differences, abs(differences)


In [5]:
# read predictions
predicitons = pd.read_csv("/content/drive/My Drive/LIN371/predictions.csv")
predicitons.head()

,Unnamed: 0,text,label,new_label,prediction
0,0,Those pussy lips need more cleaning with my to...,1,explicit_source_has_explicit_words,1
1,1,"I have choices for you. Choice seating, at that",1,explicit_source_no_explicit_words,1
2,2,I want to finish.,1,explicit_source_no_explicit_words,0
3,3,"Oh it Will, one way or an other 😉",1,explicit_source_no_explicit_words,1
4,4,"No need to thank me, thank you so much for sha...",1,explicit_source_no_explicit_words,0


In [6]:
new_predictions = pd.read_csv("/content/drive/My Drive/LIN371/predictions_mlm_tuned.csv")
new_predictions.head()

,Unnamed: 0,text,label,new_label,prediction
0,0,Those pussy lips need more cleaning with my to...,1,explicit_source_has_explicit_words,1
1,1,"I have choices for you. Choice seating, at that",1,explicit_source_no_explicit_words,1
2,2,I want to finish.,1,explicit_source_no_explicit_words,1
3,3,"Oh it Will, one way or an other 😉",1,explicit_source_no_explicit_words,1
4,4,"No need to thank me, thank you so much for sha...",1,explicit_source_no_explicit_words,1


In [7]:
# get some examples where pred is 1

pred_1 = predicitons[predicitons['prediction'] == 1].sample(5, random_state=5)
texts = pred_1['text'].tolist()
texts

['Looks wet and ready',
 'Come here you cuddly little rascal',
 'Exactly what I need after a long day at work.',
 'man i love a Bush! u are stunning',
 'Mmmmm mmmm mmm\n']

In [8]:
new_predictions[new_predictions['text'].isin(texts)]

,Unnamed: 0,text,label,new_label,prediction
128,128,Mmmmm mmmm mmm\n,1,explicit_source_no_explicit_words,1
365,365,Come here you cuddly little rascal,0,control_source_no_explicit_words,0
654,654,Looks wet and ready,1,explicit_source_no_explicit_words,1
2283,2283,Exactly what I need after a long day at work.,1,explicit_source_no_explicit_words,1
2544,2544,man i love a Bush! u are stunning,1,explicit_source_no_explicit_words,1


In [9]:
proba0, tokens0, diffs0, absdiffs0 = perturb_sent(model, tokenizer, texts[0])
proba1, tokens1, diffs1, absdiffs1 = perturb_sent(model, tokenizer, texts[1])
proba2, tokens2, diffs2, absdiffs2 = perturb_sent(model, tokenizer, texts[2])
proba3, tokens3, diffs3, absdiffs3 = perturb_sent(model, tokenizer, texts[3])
proba4, tokens4, diffs4, absdiffs4 = perturb_sent(model, tokenizer, texts[4])

sentence: Looks wet and ready
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.001     |     0.999     |       0.0        |
 looks   |     0.004     |     0.996     |      0.002       |
  wet    |     0.029     |     0.971     |      0.028       |
  and    |     0.002     |     0.998     |       0.0        |
 ready   |     0.015     |     0.985     |      0.013       |
sentence: Come here you cuddly little rascal
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.34      |     0.66      |       0.0        |
  come   |     0.682     |     0.318     |      0.342       |
  here   |     0.199     |     0.801     |      -0.14       |
  you    |     0.391     |     0.609     |      0.051       |
   cu    |     0.723     |     0.277     |      0.383       |
  ##dd   |     0.518     |     0.482     |      0.178       |
  ##ly   |     0.464     |     0.536     |      0.124       |
 little  |     0.409     |     0.591     |      0.069    

In [ ]:
# load the mlm model
mlm_model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
mlm_tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
mlm_model.to(device)
mlm_model.eval()
mlm_model.zero_grad()


In [ ]:
mlm_proba0, mlm_tokens0, mlm_diffs0, mlm_absdiffs0 = perturb_sent(mlm_model, mlm_tokenizer, texts[0])
mlm_proba1, mlm_tokens1, mlm_diffs1, mlm_absdiffs1 = perturb_sent(mlm_model, mlm_tokenizer, texts[1])
mlm_proba2, mlm_tokens2, mlm_diffs2, mlm_absdiffs2 = perturb_sent(mlm_model, mlm_tokenizer, texts[2])
mlm_proba3, mlm_tokens3, mlm_diffs3, mlm_absdiffs3 = perturb_sent(mlm_model, mlm_tokenizer, texts[3])
mlm_proba4, mlm_tokens4, mlm_diffs4, mlm_absdiffs4 = perturb_sent(mlm_model, mlm_tokenizer, texts[4])

sentence: Looks wet and ready
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.018     |     0.982     |       0.0        |
 looks   |     0.049     |     0.951     |      0.031       |
  wet    |     0.076     |     0.924     |      0.058       |
  and    |     0.045     |     0.955     |      0.027       |
 ready   |     0.16      |     0.84      |      0.142       |
sentence: Come here you cuddly little rascal
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.565     |     0.435     |       0.0        |
  come   |     0.739     |     0.261     |      0.174       |
  here   |     0.565     |     0.435     |       0.0        |
  you    |     0.649     |     0.351     |      0.084       |
   cu    |     0.584     |     0.416     |      0.018       |
  ##dd   |     0.559     |     0.441     |      -0.006      |
  ##ly   |     0.608     |     0.392     |      0.043       |
 little  |     0.682     |     0.318     |      0.117    